In [24]:
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark1")

cursor = conn.cursor()

In [25]:
df = pd.read_csv('Sales.csv')

In [26]:
df.shape

(62884, 9)

In [27]:
df.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [28]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")
df.columns

Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code'],
      dtype='object')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_number   62884 non-null  int64 
 1   line_item      62884 non-null  int64 
 2   order_date     62884 non-null  object
 3   delivery_date  13165 non-null  object
 4   customerkey    62884 non-null  int64 
 5   storekey       62884 non-null  int64 
 6   productkey     62884 non-null  int64 
 7   quantity       62884 non-null  int64 
 8   currency_code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [30]:
df['order_date']= pd.to_datetime(df['order_date'])
df['delivery_date']= pd.to_datetime(df['delivery_date'])

In [31]:
df.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD


In [32]:
df['delivery_date'] = df['delivery_date'].fillna((df['order_date'] + pd.Timedelta(days=4)))


In [33]:
df["shipping_days"] = (df['delivery_date'] - df['order_date']).dt.days

In [34]:
df.isna().sum()

order_number     0
line_item        0
order_date       0
delivery_date    0
customerkey      0
storekey         0
productkey       0
quantity         0
currency_code    0
shipping_days    0
dtype: int64

In [35]:
df.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,4
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11


In [36]:
insert_sales_values = '''
    INSERT INTO sales (
        order_number, 
        line_item, 
        order_date, 
        delivery_date, 
        customerkey, 
        storekey, 
        productkey, 
        quantity, 
        currency_code, 
        shipping_days
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the sales table
for index, row in df.iterrows():
    cursor.execute(insert_sales_values, (
        row['order_number'],
        row['line_item'],
        row['order_date'],
        row['delivery_date'],
        row['customerkey'],
        row['storekey'],
        row['productkey'],
        row['quantity'],
        row['currency_code'],
        row['shipping_days']
    ))

conn.commit()    

IntegrityError: 1062 (23000): Duplicate entry '366001' for key 'sales.PRIMARY'

In [37]:
duplicates = df[df['order_number'].duplicated()]
duplicates

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11
5,366002,3,2016-01-01,2016-01-12,266019,0,1080,4,CAD,11
7,366004,2,2016-01-01,2016-01-05,1107461,38,1529,2,GBP,4
10,366007,2,2016-01-01,2016-01-05,2035771,43,666,5,USD,4
...,...,...,...,...,...,...,...,...,...,...
62876,2243029,4,2021-02-20,2021-02-27,887764,0,580,1,EUR,7
62877,2243029,5,2021-02-20,2021-02-27,887764,0,1449,4,EUR,7
62878,2243029,6,2021-02-20,2021-02-27,887764,0,1456,2,EUR,7
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD,3
